<a href="https://colab.research.google.com/github/LuViBeBe93/tesis/blob/main/GPT2_F3_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers pandas torch bert-score nltk optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.8 MB/s eta 0:00:00


In [ ]:
# Importación de bibliotecas
from google.colab import files
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import score
import optuna
import math



In [ ]:
# Subir el archivo de datos
uploaded = files.upload()
data = pd.read_excel('BASE_30122024.xlsx')
data = pd.DataFrame(data)

# Configuración para usar GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



Saving BASE_30122024.xlsx to BASE_30122024.xlsx


In [ ]:
# Convertir datos numéricos a texto
def datos_a_texto(row):
    texto_datos = (
        f"Año: {row.get('Año', 'NaN')}, Mes: {row.get('Mes', 'NaN')}, "
        f"Trimestre: {row.get('Trimestre', 'NaN')}, Frecuencia: {row.get('Frecuencia', 'NaN')}, "
        f"Año_comparación: {row.get('Año_comparación', 'NaN')}, País: {row.get('País', 'NaN')}, "
        f"Tendencia: {row.get('Tendencia', 'NaN')}, Sector: {row.get('Sector', 'NaN')}, "
        f"Indicador: {row.get('Indicador', 'NaN')}, Valor_actual: {row.get('Valor_actual', 'NaN')}, "
        f"Variación: {row.get('Variación', 'NaN')}, Valor_comparación: {row.get('Valor_comparación', 'NaN')}, "
        f"Entidad: {row.get('Entidad', 'NaN')}"
    )
    return texto_datos

# Crear una columna con los datos numéricos convertidos a texto
data["entrada_texto"] = data.apply(datos_a_texto, axis=1)
data["texto_final"] = data["entrada_texto"] + " </s> Noticia: " + data["Noticia"]



In [ ]:
# Dataset personalizado
class NoticiasDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        texto = self.data.iloc[idx]["texto_final"]
        encoding = self.tokenizer(
            texto,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )
        input_ids = encoding["input_ids"].flatten()
        attention_mask = encoding["attention_mask"].flatten()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": input_ids,  # En GPT, las etiquetas son las mismas que la entrada
        }



In [ ]:
# Dividir datos en 80% entrenamiento y 20% evaluación
train_data, eval_data = train_test_split(data, test_size=0.2, random_state=42)



In [ ]:
# Inicializar el tokenizador y modelo GPT-2
tokenizer = AutoTokenizer.from_pretrained("datificate/gpt2-small-spanish")
model = AutoModelWithLMHead.from_pretrained("datificate/gpt2-small-spanish")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1838: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [ ]:
# Ajustar el tokenizador para manejar padding
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))



Embedding(50257, 768)

In [ ]:
# Crear datasets
train_dataset = NoticiasDataset(train_data, tokenizer)
eval_dataset = NoticiasDataset(eval_data, tokenizer)



In [ ]:
# Definir hiperparámetros iniciales
HYPERPARAMS = {
    "output_dir": "./gpt2_noticias_model",
    "save_steps": 500,
    "save_total_limit": 2,
    "logging_dir": "./logs",
    "logging_steps": 100,
}



In [ ]:
# Función para calcular BLEU en Optuna
def calcular_bleu_score(model, tokenizer, eval_dataloader):
    model.eval()
    predicciones_generadas = []
    referencias_reales = []

    with torch.no_grad():
        for batch in eval_dataloader:
            input_ids = batch["input_ids"].to(model.device)
            attention_mask = batch["attention_mask"].to(model.device)

            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=64,
                num_beams=HYPERPARAMS.get("num_beams", 5),
                no_repeat_ngram_size=HYPERPARAMS.get("no_repeat_ngram_size", 2),
                early_stopping=True,
            )
            predicciones = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            predicciones_generadas.extend(predicciones)

            referencias = tokenizer.batch_decode(batch["labels"], skip_special_tokens=True)
            referencias_reales.extend(referencias)

    # Calcular BLEU para todo el conjunto de evaluación
    smoothing_function = SmoothingFunction().method4
    bleu_scores = [
        sentence_bleu([ref.split()], pred.split(), smoothing_function=smoothing_function)
        for ref, pred in zip(referencias_reales, predicciones_generadas)
    ]
    return sum(bleu_scores) / len(bleu_scores)



In [ ]:
# Función objetivo para Optuna
def objective(trial):
    params = {
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 5e-4),
        "batch_size": trial.suggest_categorical("batch_size", [2, 4, 8]),
        "num_epochs": trial.suggest_int("num_epochs", 5, 15),
        "num_beams": trial.suggest_int("num_beams", 3, 7),
        "no_repeat_ngram_size": trial.suggest_int("no_repeat_ngram_size", 2, 3),
    }

    # Actualizar hiperparámetros
    HYPERPARAMS.update(params)

    # Crear DataLoader con el batch size seleccionado
    eval_dataloader = DataLoader(eval_dataset, batch_size=params["batch_size"], shuffle=False)

    # Configurar argumentos de entrenamiento
    training_args = TrainingArguments(
        output_dir=HYPERPARAMS["output_dir"],
        num_train_epochs=params["num_epochs"],
        per_device_train_batch_size=params["batch_size"],
        save_steps=HYPERPARAMS["save_steps"],
        save_total_limit=HYPERPARAMS["save_total_limit"],
        logging_dir=HYPERPARAMS["logging_dir"],
        logging_steps=HYPERPARAMS["logging_steps"],
        learning_rate=params["learning_rate"],
    )

    # Entrenador con los nuevos parámetros
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
    )

    # Entrenar modelo
    trainer.train()

    # Evaluar usando BLEU
    bleu_score = calcular_bleu_score(model, tokenizer, eval_dataloader)
    return bleu_score

# Crear estudio de Optuna para maximizar BLEU
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)



[I 2025-01-08 14:05:39,900] A new study created in memory with name: no-name-e7bfae28-3fb3-4c18-bc92-f90438c9885f
<ipython-input-13-31c33ecf58fd>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 5e-4),
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
100,1.649500
200,0.569300
300,0.478300
400,0.419400
500,0.396600
600,0.369700
700,0.331900
800,0.321000
900,0.298100
1000,0.303400


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

...

Step,Training Loss
100,0.084400
200,0.089900
300,0.095700
400,0.096000
500,0.097600
600,0.097100
700,0.097100
800,0.098900
900,0.097700
1000,0.099800


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Step,Training Loss
100,0.064500
200,0.067000
300,0.067200
400,0.067700
500,0.067500
600,0.067900
700,0.066400
800,0.065100
900,0.065900
1000,0.066200


Step,Training Loss
100,0.064500
200,0.067000
300,0.067200
400,0.067700
500,0.067500
600,0.067900
700,0.066400
800,0.065100
900,0.065900
1000,0.066200


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [ ]:
# Resultados finales
print("Mejores hiperparámetros:")
print(study.best_params)
print(f"Mejor BLEU obtenido: {study.best_value}")


Mejores hiperparámetros:
{'learning_rate': 2.9739423923631663e-05, 'batch_size': 4, 'num_epochs': 7, 'num_beams': 3, 'no_repeat_ngram_size': 3}
Mejor BLEU obtenido: 0.5547259232808579
